In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Импорт данных

In [18]:
# тренировочные данные
train_card_spending_df = pd.read_parquet('input_data/train_card_spending_df.parquet')
train_main_df = pd.read_parquet('input_data/train_main_df.parquet')
train_mcc_operations_df = pd.read_parquet('input_data/train_mcc_operations_df.parquet')
train_mcc_preferences_df = pd.read_parquet('input_data/train_mcc_preferences_df.parquet')
train_target = pd.read_csv('input_data/train_target.csv')

In [ ]:
# тестовые данные
test_card_spending_df = pd.read_parquet('input_data/test_card_spending_df.parquet')
test_main_df = pd.read_parquet('input_data/test_main_df.parquet')
test_mcc_operations_df = pd.read_parquet('input_data/test_mcc_operations_df.parquet')
test_mcc_preferences_df = pd.read_parquet('input_data/test_mcc_preferences_df.parquet')

# Очистка повторов id

In [19]:
# Т.к. порядок id похоже совпадает в train_target и train_main_df и других таблицах, продублируем index в отдельный столбец для merge
# будем производить merge по двум колонкам для избежания новых фейковых данных
train_target['index'] = train_target.index
train_main_df['index'] = train_main_df.index
train_card_spending_df['index'] = train_card_spending_df.index
train_mcc_operations_df['index'] = train_mcc_operations_df.index
train_mcc_preferences_df['index'] = train_mcc_preferences_df.index

train_full_info = pd.merge(train_target, train_main_df, on=['id', 'index'])

# Сценарий 1: Фантазирование всего

In [107]:
def all_nans_fullfilling(train_full_info):
    # заполнение nan в колонке происхождения авто и степени БУ авто
    train_full_info['vehicle_counrty_type_nm'] = train_full_info['vehicle_counrty_type_nm'].fillna(0)
    train_full_info['used_car_flg'] = train_full_info['used_car_flg'].fillna(0)
    train_full_info['app_vehicle_ind'] = train_full_info['app_vehicle_ind'].fillna(0)
    
    
    # идея заполнения однотипных колонок по маске
    mask_avg_dep = train_full_info.columns.str.startswith('avg_dep_avg_balance_fact_')
    mask_zp = train_full_info.columns.str.startswith('zp_')
    mask_agg = train_full_info.columns.str.startswith(('max_', 'min_', 'avg_', 'sum_'))
    mask_dep = train_full_info.columns.str.startswith('dep_')
    mask_income = train_full_info.columns.str.startswith('income_')
    mask_cnt = train_full_info.columns.str.startswith('cnt_') & ~train_full_info.columns.isin(['cnt_prolong_max', 'cnt_prolong_max_5y'])
    # заполнение медианой колонок из списка avg_dep_avg_balance_fact_...
    cols_to_fill = train_full_info.columns[mask_avg_dep]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    # заполнение медианой колонок из списка zp_...
    cols_to_fill = train_full_info.columns[mask_zp]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    # заполнение медианой колонок из списка max_..., min_..., avg_..., sum_...
    cols_to_fill = train_full_info.columns[mask_agg]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    # заполнение медианой колонок из списка dep_...
    cols_to_fill = train_full_info.columns[mask_dep]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    # заполнение медианой колонок из списка income_...
    cols_to_fill = train_full_info.columns[mask_income]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    # заполнение медианой колонок из списка cnt_...
    cols_to_fill = train_full_info.columns[mask_cnt]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    
    
    # вывод: требуется написание функции, для cnt_prolong_max которая бы ставила 0, 1, 2 на основании значения
    # dep_max_d_term (макс срок срочного вклада) и max_term (макс срок договора) для cnt_prolong_max
    # А cnt_prolong_max_5y обработать на основании cnt_prolong_max
    # Определяем границы интервалов и соответствующие метки
    bins = [0, 7, 12, 31, 32, 33, 41, 51, 58, 61, 63, 75, 80, float('inf')]
    labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    # Создаем категории для max_term
    train_full_info['max_term_category'] = pd.cut(
        train_full_info['max_term'],
        bins=bins,
        labels=labels,
        right=False
    )
    # Вычисляем медианные значения cnt_prolong_max для каждой категории
    median_values = train_full_info.groupby('max_term_category')['cnt_prolong_max'].median()
    # Заполняем пропуски в cnt_prolong_max на основе категорий max_term
    train_full_info['cnt_prolong_max'] = train_full_info['cnt_prolong_max'].fillna(
        train_full_info['max_term_category'].map(median_values))
    # Удаляем временную колонку (опционально)
    train_full_info = train_full_info.drop('max_term_category', axis=1)
    train_full_info['cnt_prolong_max_5y'] = train_full_info['cnt_prolong_max_5y'].fillna(0)
    
    
    # Вывод: app_income_app = mean(income_verified, income_verified_primary_job) + income_unverified
    func = lambda x: (x['income_verified'] + x['income_verified_primary_job']) / 2 + x['income_unverified']
    train_full_info['app_income_app'] = train_full_info.apply(func, axis=1)
    
    # Заполнение app_real_estate_ind (наличие недвижимости)
    train_full_info['app_real_estate_ind'] = train_full_info['app_real_estate_ind'].fillna('0')
    
    
    # Вывод: попробуем заполнить nan в детях и иждвенцах нулями. nan в семье возьмем как сумма детей и иждивенцев
    train_full_info['app_children_cnt'] = train_full_info['app_children_cnt'].fillna(0)
    train_full_info['app_dependent_cnt'] = train_full_info['app_dependent_cnt'].fillna(0)
    filling = train_full_info['app_children_cnt'] + train_full_info['app_dependent_cnt']
    train_full_info['app_family_cnt'] = train_full_info['app_family_cnt'].fillna(filling)

    return train_full_info

train_full_info = all_nans_fullfilling(train_full_info)


#Лучший ответ: 1.9891
    #CATBOOST:
    #iterations=2500,
    #learning_rate=0.05,
    #loss_function='RMSE',
    #eval_metric='R2',
    #random_seed=42,

C:\Temp\ipykernel_8556\1074550607.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_full_info['max_term_category'] = pd.cut(
C:\Temp\ipykernel_8556\1074550607.py:49: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_values = train_full_info.groupby('max_term_category')['cnt_prolong_max'].median()


# Сценарий 2: Удаление всего, где много nan

In [126]:
def deleting_nan_columns(train_full_info):
    # заполнение медианой колонок из списка cnt_...
    mask_cnt = train_full_info.columns.str.startswith('cnt_') & ~train_full_info.columns.isin(['cnt_prolong_max', 'cnt_prolong_max_5y'])
    cols_to_fill = train_full_info.columns[mask_cnt]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    
    mask_avg_dep = train_full_info.columns.str.startswith('avg_dep_avg_balance_fact_')
    mask_zp = train_full_info.columns.str.startswith('zp_')
    mask_agg = train_full_info.columns.str.startswith(('max_', 'min_', 'avg_', 'sum_'))
    mask_dep = train_full_info.columns.str.startswith('dep_')
    mask_income = train_full_info.columns.str.startswith('income_')
    mask_cnt = train_full_info.columns.str.startswith('cnt_')
    mask_app = train_full_info.columns.str.startswith('app_')
    
    
    # Создаем список всех колонок для удаления
    cols_to_drop = (train_full_info.columns[mask_avg_dep].tolist() +
                    train_full_info.columns[mask_zp].tolist() +
                    train_full_info.columns[mask_agg].tolist() +
                    train_full_info.columns[mask_dep].tolist() +
                    train_full_info.columns[mask_income].tolist() +
                    train_full_info.columns[mask_cnt].tolist() +
                    train_full_info.columns[mask_app].tolist()
                   )
    # Удаляем все колонки за один раз
    train_full_info = train_full_info.drop(columns=cols_to_drop)

    train_full_info = train_full_info.drop(['vehicle_counrty_type_nm'], axis=1)
    
    return train_full_info

train_full_info = deleting_nan_columns(train_full_info)


#Лучший ответ: 2.1095
    #CATBOOST:
    #iterations=1000,
    #learning_rate=0.05,
    #loss_function='RMSE',
    #eval_metric='R2',
    #random_seed=42,

# Сценарий 3: Совмещение сценария 1 и 2

In [61]:
def deleting_and_filling(train_full_info):
    train_full_info['vehicle_counrty_type_nm'] = train_full_info['vehicle_counrty_type_nm'].fillna(0)
    train_full_info['used_car_flg'] = train_full_info['used_car_flg'].fillna(0)
    train_full_info['app_vehicle_ind'] = train_full_info['app_vehicle_ind'].fillna(0)
    train_full_info['app_real_estate_ind'] = train_full_info['app_real_estate_ind'].fillna('0')

    # заполнение медианой колонок из списка cnt_... и maxterm
    mask_cnt = train_full_info.columns.str.startswith('cnt_') & ~train_full_info.columns.isin(['cnt_prolong_max', 'cnt_prolong_max_5y'])
    cols_to_fill = train_full_info.columns[mask_cnt]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    train_full_info['max_term'] = train_full_info['max_term'].fillna(train_full_info['max_term'].median())
    # заполнение медианой колонок из списка income_...
    mask_income = train_full_info.columns.str.startswith('income_')
    cols_to_fill = train_full_info.columns[mask_income]
    train_full_info[cols_to_fill] = train_full_info[cols_to_fill].fillna(train_full_info[cols_to_fill].median())
    
    # удаление колонок
    mask_avg_dep = train_full_info.columns.str.startswith('avg_dep_avg_balance_fact_')
    mask_zp = train_full_info.columns.str.startswith('zp_')
    mask_agg = (train_full_info.columns.str.startswith(('max_', 'min_', 'avg_', 'sum_'))
                & ~train_full_info.columns.isin(['max_term']))
    mask_dep = train_full_info.columns.str.startswith('dep_')
    #mask_cnt = train_full_info.columns.str.startswith('cnt_')
    # Создаем список всех колонок для удаления
    cols_to_drop = (train_full_info.columns[mask_avg_dep].tolist() +
                    train_full_info.columns[mask_zp].tolist() +
                    train_full_info.columns[mask_agg].tolist() +
                    train_full_info.columns[mask_dep].tolist()
                    #train_full_info.columns[mask_cnt].tolist()
                    )
    # Удаляем все колонки за один раз
    train_full_info = train_full_info.drop(columns=cols_to_drop)
    train_full_info = train_full_info.drop(['app_children_cnt', 'app_dependent_cnt', 'app_family_cnt', 'cnt_prolong_max_5y'], axis=1)

    # Определяем границы интервалов и соответствующие метки
    bins = [0, 7, 12, 31, 32, 33, 41, 51, 58, 61, 63, 75, 80, float('inf')]
    labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    # Создаем категории для max_term
    train_full_info['max_term_category'] = pd.cut(
        train_full_info['max_term'],
        bins=bins,
        labels=labels,
        right=False
    )
    # Вычисляем медианные значения cnt_prolong_max для каждой категории
    median_values = train_full_info.groupby('max_term_category')['cnt_prolong_max'].median()
    # Заполняем пропуски в cnt_prolong_max на основе категорий max_term
    train_full_info['cnt_prolong_max'] = train_full_info['cnt_prolong_max'].fillna(
        train_full_info['max_term_category'].map(median_values))
    # Удаляем временную колонку (опционально)
    train_full_info = train_full_info.drop('max_term_category', axis=1)
    
    # Вывод: app_income_app = mean(income_verified, income_verified_primary_job) + income_unverified
    func = lambda x: (x['income_verified'] + x['income_verified_primary_job']) / 2 + x['income_unverified']
    train_full_info['app_income_app'] = train_full_info.apply(func, axis=1)
    
    return train_full_info

train_full_info = deleting_and_filling(train_full_info)

#Лучший ответ: 2.1090
    #CATBOOST:
    #iterations=1000,
    #learning_rate=0.05,
    #loss_function='RMSE',
    #eval_metric='R2',
    #random_seed=42,

C:\Temp\ipykernel_8556\1145654557.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_values = train_full_info.groupby('max_term_category')['cnt_prolong_max'].median()


# Сценарий 4-5. Очистка вспомогательных таблиц (nan более x%) и присоединение их к основной из сценария 1 или 2

In [127]:
# ПРЕЖДЕ ВЫПОЛНИТЬ СЦЕНАРИЙ 1 ИЛИ 2

def preprocess_data(df):
    # Удаляем колонки, где NaN > x
    x = 0.5
    threshold = len(df) * 0.75
    cols_to_drop = df.columns[df.isnull().sum() > threshold]
    df = df.drop(columns=cols_to_drop)
    
    # Создаем словарь для группировки колонок по базовым названиям
    from collections import defaultdict
    col_groups = defaultdict(list)
    
    # Группируем колонки по базовым названиям (без временных суффиксов)
    for col in df.columns:
        base_col = col
        for suffix in ['_1m', '_3m', '_6m', '_12m', '_1', '_3', '_6', '_12', '_7d']:
            if col.endswith(suffix):
                base_col = col[:-len(suffix)]
                break
        col_groups[base_col].append(col)
    
    # Выбираем какие колонки оставить (приоритет: _1m, затем _1, затем _12m/_12)
    cols_to_keep = set()
    cols_to_remove = set()
    
    for base_col, variants in col_groups.items():
        if len(variants) > 1:  # Если есть временные варианты
            # Проверяем наличие предпочтительных вариантов
            preferred = [v for v in variants if v.endswith(('_1m', '_1'))]
            if not preferred:
                preferred = [v for v in variants if v.endswith(('_12m', '_12'))]
            
            if preferred:
                # Оставляем первый предпочтительный вариант
                cols_to_keep.add(preferred[0])
                # Остальные добавляем на удаление
                for v in variants:
                    if v != preferred[0]:
                        cols_to_remove.add(v)
    
    # Удаляем колонки с другими периодами
    df = df.drop(columns=cols_to_remove)
    
    return df

# Обработка каждой таблицы
train_card_spending_df = preprocess_data(train_card_spending_df)
train_mcc_operations_df = preprocess_data(train_mcc_operations_df)
train_mcc_preferences_df = preprocess_data(train_mcc_preferences_df)

# Обработка каждой таблицы
train_card_spending_df = preprocess_data(train_card_spending_df)
train_mcc_operations_df = preprocess_data(train_mcc_operations_df)
train_mcc_preferences_df = preprocess_data(train_mcc_preferences_df)

In [128]:
# Добавить для сценария 4
train_full_info = pd.merge(train_full_info, train_card_spending_df, on=['id', 'index'])
train_full_info = pd.merge(train_full_info, train_mcc_operations_df, on=['id', 'index'])
train_full_info = pd.merge(train_full_info, train_mcc_preferences_df, on=['id', 'index'])

#Лучший ответ: 1.9990
    # Сценарий 1 + очистка nan у вспомогательных таблиц - более 75% и более 50%
    #iterations=2500,
    #learning_rate=0.05,
    #loss_function='RMSE',
    #eval_metric='R2',
    #random_seed=42,

# Сценарий 6

In [20]:
train_card_spending_df = pd.merge(train_target, train_card_spending_df, on=['id', 'index'])
train_mcc_operations_df = pd.merge(train_target, train_mcc_operations_df, on=['id', 'index'])
train_mcc_preferences_df = pd.merge(train_target, train_mcc_preferences_df, on=['id', 'index'])

In [ ]:
corr_matrix = train_card_spending_df.select_dtypes(include=['number']).corr()
corr_matrix_dea = corr_matrix.loc[['target'], :]
corr_matrix_dea.loc[:, (corr_matrix_dea.abs() > 0.4).any(axis = 0)]

In [21]:
def filter_by_correlation(df, target_col='target', correlation_threshold=0.4):
    """
    Фильтрует колонки датафрейма по корреляции с целевой переменной,
    затем удаляет дублирующие временные колонки.
    
    Параметры:
    - df: исходный датафрейм
    - target_col: название целевой колонки
    - correlation_threshold: порог корреляции для отбора колонок
    
    Возвращает:
    - Отфильтрованный датафрейм
    """
    # Проверяем, что целевая колонка существует
    if target_col not in df.columns:
        raise ValueError(f"Целевая колонка '{target_col}' не найдена в датафрейме")
    
    # Вычисляем корреляцию только для числовых колонок
    numeric_df = df.select_dtypes(include=['number'])
    
    # Получаем матрицу корреляций с целевой колонкой
    corr_matrix = numeric_df.corr()
    target_corr = corr_matrix.loc[[target_col], :]
    
    # Выбираем колонки с корреляцией выше порога (по модулю)
    significant_cols = target_corr.columns[(target_corr.abs() > correlation_threshold).any(axis=0)]
    
    # Оставляем только значимые колонки + саму целевую
    filtered_df = df[list(significant_cols) + [target_col]]
    
    # Теперь применяем обработку похожих колонок (как в предыдущем решении)
    def remove_temporal_duplicates(temp_df):
        """Вспомогательная функция для удаления временных дубликатов"""
        from collections import defaultdict
        col_groups = defaultdict(list)
        
        # Группируем колонки по базовым названиям
        for col in temp_df.columns:
            if col == target_col:
                continue  # Целевую колонку не трогаем
                
            base_col = col
            for suffix in ['_1m', '_3m', '_6m', '_12m', '_1', '_3', '_6', '_12', '_7d']:
                if col.endswith(suffix):
                    base_col = col[:-len(suffix)]
                    break
            col_groups[base_col].append(col)
        
        cols_to_keep = set()
        cols_to_remove = set()
        
        for base_col, variants in col_groups.items():
            if len(variants) > 1:  # Если есть временные варианты
                # Приоритет: _1m, затем _1, затем _12m/_12
                preferred = [v for v in variants if v.endswith(('_1m', '_1'))] or \
                           [v for v in variants if v.endswith(('_12m', '_12'))]
                
                if preferred:
                    cols_to_keep.add(preferred[0])
                    for v in variants:
                        if v != preferred[0]:
                            cols_to_remove.add(v)
        
        # Удаляем дубликаты, оставляя целевую колонку
        return temp_df.drop(columns=cols_to_remove)
    
    # Применяем функцию удаления дубликатов
    final_df = remove_temporal_duplicates(filtered_df)
    
    return final_df

# Пример использования:
# processed_df = filter_by_correlation(train_full_info, target_col='target', correlation_threshold=0.4)

In [22]:
dataset_list = [train_full_info, train_card_spending_df, train_mcc_operations_df, train_mcc_preferences_df]

for dataset in dataset_list:
    dataset = filter_by_correlation(dataset)

KeyboardInterrupt: 

# Проверка на заполнение пропусков

In [129]:
# Проверка на отсуствие nan
nan_count = train_full_info.isna().sum()
nan_count = nan_count[nan_count != 0]
nan_count.head(30)

used_car_flg                        31299
cat_maxspend_v1_1                   48985
cat_maxspend_v2_1                   49348
cc_avg_trns_1m                      26877
cc_cnt_trns_12m                     26877
cc_sum_trns_12m                     26877
cnt_mcc_codes_1                     42301
cnt_tr_abroad_1                     42301
cnt_tr_airlines_1                   42301
cnt_tr_alkochol_1m                  42301
cnt_tr_all_1                        40416
cnt_tr_appliance_retail_1           42301
cnt_tr_auto_rental_1                42301
cnt_tr_auto_repair_1m               42301
cnt_tr_auto_services_1              42301
cnt_tr_books_store_1                42301
cnt_tr_business_services_1          42301
cnt_tr_buy_1m                       42301
cnt_tr_cash_1                       42301
cnt_tr_cash_in_1                    40416
cnt_tr_cash_services_1              42301
cnt_tr_charity_1                    42301
cnt_tr_cleaning_services_1          42301
cnt_tr_computer_program_retail_1  

# Обработка категориальных данных

In [ ]:
train_full_info.select_dtypes(include=['object']).columns

In [ ]:
train_full_info['savings_service_model_cd'].unique()

In [ ]:
# Удалим две наиболее разносторонние категориальные колонки
train_full_info = train_full_info.drop(['brand_nm', 'industry_nm'], axis=1)

In [ ]:
cat_columns = train_full_info.select_dtypes(include=['object']).columns

train_encoded = pd.get_dummies(
    train_full_info,
    columns=cat_columns,
    prefix=cat_columns,
    drop_first=True  # избегаем дамми-ловушку
).astype('float64')

test_encoded = pd.get_dummies(
    #train_full_info,
    columns=cat_columns,
    prefix=cat_columns,
    drop_first=True  # избегаем дамми-ловушку
).astype('float64')

In [ ]:
train_encoded

# Экспорт

In [130]:
train_full_info.to_csv('processed_data.csv', index=False)

#test_full_info.to_csv('test_data.csv', index=False)